# 🎯 Day 3 조별 프로젝트: E2E MLOps Pipeline

## 📋 개요

3일간 학습한 MLOps 기술을 종합하여 **완전한 E2E ML 파이프라인**을 구축합니다.

### 평가 기준

| 항목 | 배점 | 기준 |
|------|------|------|
| Kubeflow Pipeline | 40점 | 최소 5개 컴포넌트, Succeeded 상태 |
| MLflow Tracking | 20점 | 최소 2회 Run, 파라미터/메트릭 기록 |
| Feature Engineering | 10점 | 1개 이상 파생 피처 생성 |
| KServe 배포 (선택) | 25점 | InferenceService 생성 및 API 테스트 |

---

### 파이프라인 구조

```
┌──────────────┐     ┌──────────────┐     ┌──────────────┐
│  1. Load     │ ──▶ │  2. Preprocess│ ──▶ │  3. Feature  │
│     Data     │     │              │     │   Engineering│
└──────────────┘     └──────────────┘     └──────────────┘
                                                  │
                                                  ▼
┌──────────────┐     ┌──────────────┐     ┌──────────────┐
│  6. Deploy   │ ◀── │  5. Evaluate │ ◀── │  4. Train    │
│   (KServe)   │     │              │     │     Model    │
└──────────────┘     └──────────────┘     └──────────────┘
```

---

## 1. 환경 설정

In [ ]:
# 패키지 설치
!pip install kfp==2.7.0 mlflow>=2.9.0 scikit-learn pandas numpy kubernetes -q
print("✅ 패키지 설치 완료!")

In [ ]:
# Import
import os
from kfp import dsl
from kfp.dsl import component, Input, Output, Dataset, Model, Metrics
from kfp import compiler
import kfp

print(f"KFP Version: {kfp.__version__}")

In [ ]:
# ============================================================
# ⚠️ TODO: 팀 설정으로 변경하세요!
# ============================================================
TEAM_NAME = "team-XX"                        # 예: team-01, team-02, ..., team-06

# 네임스페이스 자동 감지 또는 환경 변수 기반 설정
def get_namespace():
    try:
        with open('/var/run/secrets/kubernetes.io/serviceaccount/namespace', 'r') as f:
            return f.read().strip()
    except FileNotFoundError:
        user_num = os.getenv('USER_NUM', '01')
        return f"kubeflow-user{user_num}"

USER_NAMESPACE = get_namespace()

# MLflow 설정 (변경 불필요)
MLFLOW_TRACKING_URI = "http://mlflow-server-service.mlflow-system.svc.cluster.local:5000"

print("=" * 60)
print("  팀 설정")
print("=" * 60)
print(f"  Team: {TEAM_NAME}")
print(f"  Namespace: {USER_NAMESPACE}")
print(f"  MLflow URI: {MLFLOW_TRACKING_URI}")
print("=" * 60)

---

## 2. 컴포넌트 정의

### 2.1 Component 1: 데이터 로드

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "scikit-learn==1.3.2"]
)
def load_data(
    data_source: str,
    output_data: Output[Dataset]
):
    """
    California Housing 데이터셋 로드
    
    Args:
        data_source: 데이터 소스 (sklearn)
        output_data: 출력 데이터셋 경로
    """
    import pandas as pd
    from sklearn.datasets import fetch_california_housing
    
    print("=" * 60)
    print("  Step 1: Load Data")
    print("=" * 60)
    
    # California Housing 데이터셋 로드
    housing = fetch_california_housing(as_frame=True)
    df = housing.frame
    
    print(f"  Source: {data_source}")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {list(df.columns)}")
    print(f"  Target: MedHouseVal (Median House Value in $100k)")
    
    # CSV로 저장
    df.to_csv(output_data.path, index=False)
    print(f"  ✅ Data saved: {output_data.path}")

print("✅ load_data 컴포넌트 정의 완료!")

### 2.2 Component 2: 전처리

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "scikit-learn==1.3.2", "numpy==1.24.3"]
)
def preprocess(
    input_data: Input[Dataset],
    X_train_out: Output[Dataset],
    X_test_out: Output[Dataset],
    y_train_out: Output[Dataset],
    y_test_out: Output[Dataset],
    test_size: float = 0.2
):
    """
    데이터 전처리: Train/Test 분할 및 정규화
    
    Args:
        input_data: 입력 데이터셋
        X_train_out, X_test_out: 피처 출력
        y_train_out, y_test_out: 타겟 출력
        test_size: 테스트 데이터 비율
    """
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    
    print("=" * 60)
    print("  Step 2: Preprocess")
    print("=" * 60)
    
    # 데이터 로드
    df = pd.read_csv(input_data.path)
    print(f"  Loaded {len(df)} rows")
    
    # 피처와 타겟 분리
    X = df.drop(columns=['MedHouseVal'])
    y = df['MedHouseVal']
    
    # Train/Test 분할
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42
    )
    
    # StandardScaler로 정규화
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(
        scaler.fit_transform(X_train),
        columns=X_train.columns
    )
    X_test_scaled = pd.DataFrame(
        scaler.transform(X_test),
        columns=X_test.columns
    )
    
    # 저장
    X_train_scaled.to_csv(X_train_out.path, index=False)
    X_test_scaled.to_csv(X_test_out.path, index=False)
    y_train.to_csv(y_train_out.path, index=False)
    y_test.to_csv(y_test_out.path, index=False)
    
    print(f"  Train: {len(X_train)} samples")
    print(f"  Test: {len(X_test)} samples")
    print(f"  ✅ Preprocessing completed")

print("✅ preprocess 컴포넌트 정의 완료!")

### 2.3 Component 3: 피처 엔지니어링

⚠️ **TODO: 최소 1개 이상의 파생 피처를 추가하세요!**

#### 피처 아이디어:
```python
# 1. 침실 비율
df['bedroom_ratio'] = df['AveBedrms'] / (df['AveRooms'] + 1e-6)

# 2. 인당 방 수
df['rooms_per_person'] = df['AveRooms'] / (df['AveOccup'] + 1e-6)

# 3. 밀집도
df['density'] = df['Population'] * df['AveOccup']

# 4. 소득 × 방 수 상호작용
df['income_rooms'] = df['MedInc'] * df['AveRooms']

# 5. Bay Area까지 거리
df['dist_to_bay'] = np.sqrt(df['Latitude']**2 + df['Longitude']**2)
```

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "numpy==1.24.3"]
)
def feature_engineering(
    X_train_in: Input[Dataset],
    X_test_in: Input[Dataset],
    X_train_out: Output[Dataset],
    X_test_out: Output[Dataset]
) -> int:
    """
    피처 엔지니어링: 파생 변수 생성
    
    Returns:
        int: 생성된 피처 수
    """
    import pandas as pd
    import numpy as np
    
    print("=" * 60)
    print("  Step 3: Feature Engineering")
    print("=" * 60)
    
    # 데이터 로드
    X_train = pd.read_csv(X_train_in.path)
    X_test = pd.read_csv(X_test_in.path)
    
    original_cols = list(X_train.columns)
    print(f"  Original features: {len(original_cols)}")
    
    def add_features(df):
        """파생 변수 추가"""
        df = df.copy()
        
        # ============================================================
        # ⚠️ TODO: 여기에 파생 변수를 추가하세요! (최소 1개 이상)
        # ============================================================
        
        # 예시 1: 침실 비율
        df['bedroom_ratio'] = df['AveBedrms'] / (df['AveRooms'] + 1e-6)
        
        # 예시 2: TODO - 추가 피처를 구현하세요!
        # df['rooms_per_person'] = ...
        
        # 예시 3: TODO - 추가 피처를 구현하세요!
        # df['density'] = ...
        
        # ============================================================
        
        return df
    
    # 피처 추가
    X_train_fe = add_features(X_train)
    X_test_fe = add_features(X_test)
    
    # 새로 생성된 피처 확인
    new_cols = [c for c in X_train_fe.columns if c not in original_cols]
    print(f"  New features ({len(new_cols)}):")
    for feat in new_cols:
        stats = X_train_fe[feat].describe()
        print(f"    - {feat}: mean={stats['mean']:.4f}, std={stats['std']:.4f}")
    
    print(f"  Total features: {len(X_train_fe.columns)}")
    
    # 저장
    X_train_fe.to_csv(X_train_out.path, index=False)
    X_test_fe.to_csv(X_test_out.path, index=False)
    
    print(f"  ✅ Feature engineering completed")
    
    return len(new_cols)

print("✅ feature_engineering 컴포넌트 정의 완료!")

### 2.4 Component 4: 모델 학습

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=[
        "pandas==2.0.3",
        "scikit-learn==1.3.2",
        "mlflow==2.9.2",
        "numpy==1.24.3",
        "boto3"
    ]
)
def train_model(
    X_train: Input[Dataset],
    X_test: Input[Dataset],
    y_train: Input[Dataset],
    y_test: Input[Dataset],
    mlflow_tracking_uri: str,
    experiment_name: str,
    team_name: str,
    n_estimators: int = 100,
    max_depth: int = 10
) -> str:
    """
    모델 학습 및 MLflow 기록
    
    Returns:
        str: MLflow Run ID
    """
    import pandas as pd
    import numpy as np
    import mlflow
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    import os
    
    print("=" * 60)
    print(f"  Step 4: Train Model - {team_name}")
    print("=" * 60)
    
    # 데이터 로드
    X_train_df = pd.read_csv(X_train.path)
    X_test_df = pd.read_csv(X_test.path)
    y_train_df = pd.read_csv(y_train.path)
    y_test_df = pd.read_csv(y_test.path)
    
    print(f"  Training data: {X_train_df.shape}")
    print(f"  Test data: {X_test_df.shape}")
    
    # MLflow 설정
    os.environ['MLFLOW_TRACKING_URI'] = mlflow_tracking_uri
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    mlflow.set_experiment(experiment_name)
    
    with mlflow.start_run(run_name=f"{team_name}-run") as run:
        run_id = run.info.run_id
        print(f"  MLflow Run ID: {run_id}")
        
        # 파라미터 로깅
        mlflow.log_params({
            "n_estimators": n_estimators,
            "max_depth": max_depth,
            "random_state": 42,
            "n_features": X_train_df.shape[1]
        })
        mlflow.set_tag("team", team_name)
        
        # 모델 학습
        print(f"  Training RandomForest (n_estimators={n_estimators}, max_depth={max_depth})...")
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42,
            n_jobs=-1
        )
        model.fit(X_train_df, y_train_df.values.ravel())
        
        # 예측 및 평가
        y_pred = model.predict(X_test_df)
        
        r2 = r2_score(y_test_df, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test_df, y_pred))
        mae = mean_absolute_error(y_test_df, y_pred)
        
        # 메트릭 로깅
        mlflow.log_metrics({
            "r2": r2,
            "rmse": rmse,
            "mae": mae
        })
        
        print(f"  Performance:")
        print(f"    - R2 Score: {r2:.4f}")
        print(f"    - RMSE: {rmse:.4f}")
        print(f"    - MAE: {mae:.4f}")
        
        # 피처 중요도 상위 5개
        feature_importance = dict(zip(
            X_train_df.columns,
            model.feature_importances_
        ))
        sorted_importance = sorted(
            feature_importance.items(),
            key=lambda x: x[1],
            reverse=True
        )[:5]
        
        print(f"  Top 5 Feature Importance:")
        for feat, imp in sorted_importance:
            print(f"    - {feat}: {imp:.4f}")
        
        # 모델 저장
        mlflow.sklearn.log_model(model, "model")
        
        print(f"  ✅ Training completed")
    
    return run_id

print("✅ train_model 컴포넌트 정의 완료!")

### 2.5 Component 5: 모델 평가

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["mlflow==2.9.2"]
)
def evaluate_model(
    run_id: str,
    mlflow_tracking_uri: str,
    r2_threshold: float = 0.75
) -> str:
    """
    모델 평가 및 배포 결정
    
    Returns:
        str: "deploy" 또는 "skip"
    """
    import mlflow
    import os
    
    print("=" * 60)
    print("  Step 5: Evaluate Model")
    print("=" * 60)
    
    # MLflow 설정
    os.environ['MLFLOW_TRACKING_URI'] = mlflow_tracking_uri
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    
    # 메트릭 조회
    client = mlflow.tracking.MlflowClient()
    run = client.get_run(run_id)
    
    r2 = float(run.data.metrics.get("r2", 0))
    rmse = float(run.data.metrics.get("rmse", 0))
    mae = float(run.data.metrics.get("mae", 0))
    
    print(f"  Run ID: {run_id}")
    print(f"  Metrics:")
    print(f"    - R2 Score: {r2:.4f}")
    print(f"    - RMSE: {rmse:.4f}")
    print(f"    - MAE: {mae:.4f}")
    print(f"  Threshold: R2 >= {r2_threshold}")
    
    # 배포 결정
    if r2 >= r2_threshold:
        decision = "deploy"
        print(f"  ✅ Decision: DEPLOY (R2 >= threshold)")
    else:
        decision = "skip"
        print(f"  ⚠️ Decision: SKIP (R2 < threshold)")
    
    # 결정 태그 저장
    with mlflow.start_run(run_id=run_id):
        mlflow.set_tag("deployment_decision", decision)
    
    return decision

print("✅ evaluate_model 컴포넌트 정의 완료!")

### 2.6 Component 6: 모델 배포 (KServe) - 선택사항

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["kubernetes==28.1.0"]
)
def deploy_model(
    run_id: str,
    model_name: str,
    namespace: str
):
    """
    KServe InferenceService로 모델 배포
    
    Note: 이 컴포넌트는 선택사항입니다 (25점)
    """
    from kubernetes import client, config
    from kubernetes.client.rest import ApiException
    import time
    
    print("=" * 60)
    print("  Step 6: Deploy Model (KServe)")
    print("=" * 60)
    
    print(f"  Model Name: {model_name}")
    print(f"  Namespace: {namespace}")
    print(f"  Run ID: {run_id}")
    
    # Kubernetes 클라이언트 설정
    try:
        config.load_incluster_config()
    except:
        config.load_kube_config()
    
    api = client.CustomObjectsApi()
    
    # InferenceService 정의
    isvc = {
        "apiVersion": "serving.kserve.io/v1beta1",
        "kind": "InferenceService",
        "metadata": {
            "name": model_name,
            "namespace": namespace,
            "annotations": {
                "sidecar.istio.io/inject": "false"
            }
        },
        "spec": {
            "predictor": {
                "sklearn": {
                    "storageUri": f"mlflow-artifacts:/{run_id}/model",
                    "resources": {
                        "requests": {"cpu": "100m", "memory": "256Mi"},
                        "limits": {"cpu": "500m", "memory": "512Mi"}
                    }
                }
            }
        }
    }
    
    # 기존 InferenceService 삭제 (있는 경우)
    try:
        api.delete_namespaced_custom_object(
            group="serving.kserve.io",
            version="v1beta1",
            namespace=namespace,
            plural="inferenceservices",
            name=model_name
        )
        print(f"  Deleted existing InferenceService")
        time.sleep(5)
    except ApiException as e:
        if e.status != 404:
            raise
    
    # 새 InferenceService 생성
    api.create_namespaced_custom_object(
        group="serving.kserve.io",
        version="v1beta1",
        namespace=namespace,
        plural="inferenceservices",
        body=isvc
    )
    
    print(f"  ✅ InferenceService created")
    
    # 배포 상태 확인 (최대 60초)
    print(f"  Waiting for deployment...")
    for i in range(6):
        time.sleep(10)
        try:
            status = api.get_namespaced_custom_object(
                group="serving.kserve.io",
                version="v1beta1",
                namespace=namespace,
                plural="inferenceservices",
                name=model_name
            )
            conditions = status.get("status", {}).get("conditions", [])
            ready = next((c for c in conditions if c.get("type") == "Ready"), None)
            if ready and ready.get("status") == "True":
                print(f"  ✅ InferenceService READY!")
                break
            print(f"  ⏳ Status: {ready.get('status') if ready else 'Unknown'} ({(i+1)*10}s)")
        except Exception as e:
            print(f"  ⚠️ Error checking status: {e}")
    
    print(f"  Endpoint: http://{model_name}.{namespace}.svc.cluster.local/v1/models/{model_name}:predict")
    print(f"  ✅ Deployment completed")

print("✅ deploy_model 컴포넌트 정의 완료!")

### 2.7 Component 7: 알림 (성능 미달 시)

In [ ]:
@component(base_image="python:3.9-slim")
def send_alert(
    run_id: str,
    team_name: str
):
    """
    성능 미달 알림
    """
    print("=" * 60)
    print(f"  Alert - {team_name}")
    print("=" * 60)
    print(f"  ⚠️ Model did not meet performance threshold")
    print(f"  Run ID: {run_id}")
    print(f"  Recommendations:")
    print(f"    1. Add more features")
    print(f"    2. Tune hyperparameters")
    print(f"    3. Try different algorithms")

print("✅ send_alert 컴포넌트 정의 완료!")

---

## 3. 파이프라인 정의

In [ ]:
@dsl.pipeline(
    name=f"{TEAM_NAME}-pipeline",
    description=f"E2E ML Pipeline by {TEAM_NAME}"
)
def project_pipeline(
    data_source: str = "sklearn",
    team_name: str = TEAM_NAME,
    experiment_name: str = f"{TEAM_NAME}-experiment",
    model_name: str = f"{TEAM_NAME}-model",
    namespace: str = USER_NAMESPACE,
    mlflow_tracking_uri: str = MLFLOW_TRACKING_URI,
    n_estimators: int = 100,
    max_depth: int = 10,
    r2_threshold: float = 0.75
):
    """
    E2E ML Pipeline
    
    Flow:
    1. Load Data
    2. Preprocess
    3. Feature Engineering
    4. Train Model (with MLflow)
    5. Evaluate Model
    6. Deploy (if R2 >= threshold) or Alert (if R2 < threshold)
    """
    
    # Step 1: 데이터 로드
    load_task = load_data(data_source=data_source)
    
    # Step 2: 전처리
    preprocess_task = preprocess(
        input_data=load_task.outputs["output_data"]
    )
    
    # Step 3: 피처 엔지니어링
    feature_task = feature_engineering(
        X_train_in=preprocess_task.outputs["X_train_out"],
        X_test_in=preprocess_task.outputs["X_test_out"]
    )
    
    # Step 4: 모델 학습
    train_task = train_model(
        X_train=feature_task.outputs["X_train_out"],
        X_test=feature_task.outputs["X_test_out"],
        y_train=preprocess_task.outputs["y_train_out"],
        y_test=preprocess_task.outputs["y_test_out"],
        mlflow_tracking_uri=mlflow_tracking_uri,
        experiment_name=experiment_name,
        team_name=team_name,
        n_estimators=n_estimators,
        max_depth=max_depth
    )
    
    # Step 5: 평가
    evaluate_task = evaluate_model(
        run_id=train_task.output,
        mlflow_tracking_uri=mlflow_tracking_uri,
        r2_threshold=r2_threshold
    )
    
    # Step 6: 조건부 배포 또는 알림
    with dsl.If(evaluate_task.output == "deploy"):
        deploy_model(
            run_id=train_task.output,
            model_name=model_name,
            namespace=namespace
        )
    
    with dsl.If(evaluate_task.output == "skip"):
        send_alert(
            run_id=train_task.output,
            team_name=team_name
        )

print("✅ project_pipeline 정의 완료!")

---

## 4. 파이프라인 컴파일

In [ ]:
# 파이프라인 컴파일
pipeline_file = f"{TEAM_NAME}_pipeline.yaml"

compiler.Compiler().compile(
    pipeline_func=project_pipeline,
    package_path=pipeline_file
)

print("=" * 60)
print("  파이프라인 컴파일 완료!")
print("=" * 60)
print(f"  파일: {pipeline_file}")
print(f"  팀: {TEAM_NAME}")
print("")
print("  다음 단계:")
print("  1. Kubeflow Dashboard → Pipelines → Upload pipeline")
print(f"  2. {pipeline_file} 업로드")
print("  3. Create run → Start")
print("=" * 60)

---

## 5. 파이프라인 실행 (SDK 방식 - 선택사항)

UI 대신 SDK로 직접 실행할 수 있습니다.

In [ ]:
# SDK로 파이프라인 실행 (선택사항)
# UI에서 실행하는 것을 권장합니다.

# from kfp.client import Client
# 
# client = Client()
# 
# run = client.create_run_from_pipeline_func(
#     pipeline_func=project_pipeline,
#     arguments={
#         "team_name": TEAM_NAME,
#         "experiment_name": f"{TEAM_NAME}-experiment",
#         "model_name": f"{TEAM_NAME}-model",
#         "namespace": USER_NAMESPACE,
#         "n_estimators": 100,
#         "max_depth": 10,
#         "r2_threshold": 0.75
#     },
#     experiment_name=f"{TEAM_NAME}-experiment"
# )
# 
# print(f"Run ID: {run.run_id}")

print("💡 SDK 실행 코드는 주석 처리되어 있습니다.")
print("   UI에서 파이프라인을 업로드하고 실행하세요.")

---

## 6. 결과 확인

### 6.1 MLflow UI에서 확인

```
http://mlflow-server-service.mlflow-system.svc.cluster.local:5000
```

확인 항목:
- Experiment: `{TEAM_NAME}-experiment`
- Run: `{TEAM_NAME}-run`
- Parameters: n_estimators, max_depth, n_features
- Metrics: r2, rmse, mae
- Artifacts: model

### 6.2 KServe 배포 확인 (선택사항)

In [ ]:
# KServe InferenceService 상태 확인
!kubectl get inferenceservices -n {USER_NAMESPACE}

In [ ]:
# API 테스트 (KServe 배포 완료 후)
import json

# 테스트 데이터 (8개 + 파생 피처)
test_input = {
    "instances": [
        [0.5, 0.3, 0.2, 0.1, -0.2, 0.4, 0.1, -0.5, 0.15]  # 9개 피처 (파생 피처 1개 포함)
    ]
}

print("테스트 입력:")
print(json.dumps(test_input, indent=2))
print("")
print("curl 명령어:")
print(f"curl -X POST http://{TEAM_NAME}-model.{USER_NAMESPACE}.svc.cluster.local/v1/models/{TEAM_NAME}-model:predict \\")
print(f"  -H 'Content-Type: application/json' \\")
print(f"  -d '{json.dumps(test_input)}'")

---

## ✅ 체크리스트

### 파이프라인 구현
- [ ] 팀 설정 변경 완료 (TEAM_NAME)
- [ ] Feature Engineering에 최소 1개 피처 추가
- [ ] 파이프라인 컴파일 성공
- [ ] YAML 파일 생성 확인

### 파이프라인 실행
- [ ] Kubeflow UI에 파이프라인 업로드
- [ ] Run 생성 및 실행
- [ ] 모든 컴포넌트 Succeeded 상태

### 결과 확인
- [ ] MLflow에서 Experiment/Run 확인
- [ ] 파라미터/메트릭 기록 확인
- [ ] (선택) KServe InferenceService Ready 상태

### 발표 준비
- [ ] 시연 화면 준비 (Kubeflow UI, MLflow UI)
- [ ] 코드 설명 준비
- [ ] Q&A 예상 질문 준비

---

## 🎉 수고하셨습니다!

문제가 있으면 손들어 주세요. 강사가 도와드리겠습니다.

---

**현대오토에버 MLOps Training**